In [1]:
import flair

/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:

flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
s = flair.data.Sentence(txt)
flair_sentiment.predict(s)
total_sentiment = s.labels
total_sentiment
s.tag, s.score
flair_sentiment.predict(s);
s.labels
# from flair.data_fetcher import NLPTask
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

sec_corpus = flair.datasets.NER_ENGLISH_SEC_FILLINGS()
print(sec_corpus.obtain_statistics())
word_embeddings = [WordEmbeddings('glove'),
                   FlairEmbeddings('news-forward'),
                   FlairEmbeddings('news-backward'),
                   ]
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings  #, Sentence

glove_embedding = WordEmbeddings('glove')
flair_embedding_forward = FlairEmbeddings('news-forward')
flair_embedding_backward = FlairEmbeddings('news-backward')

document_embeddings = DocumentPoolEmbeddings([glove_embedding,
                                              flair_embedding_backward,
                                              flair_embedding_backward],
                                             mode='min')
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                   hidden_size=512,
                                                                   reproject_words=True,
                                                                   reproject_words_dimension=256,
                                                                   )

classifier = TextClassifier(document_embeddings, label_type=, multi_label=False)

from flair.embeddings import WordEmbeddings, DocumentRNNEmbeddings

glove_embedding = WordEmbeddings('glove')

document_embeddings = DocumentRNNEmbeddings([glove_embedding])
sentence = Sentence('The grass is green . And the sky is blue .')

document_embeddings.embed(sentence)

print(sentence.get_embedding())
df

In [4]:
import flair
from flair.datasets import UD_ENGLISH
from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

In [5]:
corpus = flair.datasets.NER_ENGLISH_SEC_FILLINGS()

2022-05-30 20:23:39,192 https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/SEC-filings/CONLL-format/data/test/FIN3.txt not found in cache, downloading to /var/folders/kw/xkx354ds5q76y81wmz9s5w980000gn/T/tmprlbxgihj


167860B [00:00, 26733591.64B/s]                                                                                         

2022-05-30 20:23:39,316 copying /var/folders/kw/xkx354ds5q76y81wmz9s5w980000gn/T/tmprlbxgihj to cache at /Users/etiennebruno/.flair/datasets/ner_english_sec_fillings/FIN3.txt
2022-05-30 20:23:39,317 removing temp file /var/folders/kw/xkx354ds5q76y81wmz9s5w980000gn/T/tmprlbxgihj


2022-05-30 20:23:39,584 https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/SEC-filings/CONLL-format/data/train/FIN5.txt not found in cache, downloading to /var/folders/kw/xkx354ds5q76y81wmz9s5w980000gn/T/tmpu1nhhhyo


523060B [00:00, 24424669.35B/s]                                                                                         

2022-05-30 20:23:39,773 copying /var/folders/kw/xkx354ds5q76y81wmz9s5w980000gn/T/tmpu1nhhhyo to cache at /Users/etiennebruno/.flair/datasets/ner_english_sec_fillings/FIN5.txt
2022-05-30 20:23:39,776 removing temp file /var/folders/kw/xkx354ds5q76y81wmz9s5w980000gn/T/tmpu1nhhhyo
2022-05-30 20:23:39,779 Reading data from /Users/etiennebruno/.flair/datasets/ner_english_sec_fillings
2022-05-30 20:23:39,784 Train: /Users/etiennebruno/.flair/datasets/ner_english_sec_fillings/FIN5.txt
2022-05-30 20:23:39,786 Dev: None
2022-05-30 20:23:39,786 Test: /Users/etiennebruno/.flair/datasets/ner_english_sec_fillings/FIN3.txt


In [6]:
label_type = 'pos'

In [7]:
label_dict = corpus.make_label_dictionary(label_type=label_type)

2022-05-30 20:23:42,215 Computing label dictionary. Progress:


1051it [00:00, 20261.87it/s]

2022-05-30 20:23:42,282 Dictionary created for label 'pos' with 42 values: NNP (seen 5586 times), NN (seen 5105 times), IN (seen 4365 times), DT (seen 3811 times), NNS (seen 2114 times), CC (seen 2096 times), , (seen 1883 times), JJ (seen 1732 times), CD (seen 1366 times), : (seen 1318 times), . (seen 1232 times), VBN (seen 947 times), VB (seen 934 times), TO (seen 896 times), VBZ (seen 680 times), RB (seen 629 times), MD (seen 439 times), VBG (seen 430 times), VBD (seen 330 times), VBP (seen 312 times)


In [16]:
# 4. initialize embeddings
embedding_types = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
     CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
     FlairEmbeddings('news-forward'),
     FlairEmbeddings('news-backward'),
]

NameError: name 'CharacterEmbeddings' is not defined

In [ ]:
embeddings = StackedEmbeddings(embeddings=embedding_types)

In [ ]:
# 5. initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

In [14]:
# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

In [15]:
# 7. start training
trainer.train('resources/taggers/example-upos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

2022-05-30 20:42:57,202 ----------------------------------------------------------------------------------------------------
2022-05-30 20:42:57,203 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=44, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2022-05-30 20:42:57,203 ----------------------------------------------------------------------------------------------------
2022-05-30 20:42:57,204 Corpus: "Corpus: 1051 train + 117 dev + 305 test sentences"
2022-05-30 20:42:57,204 ----------------------------------------------------------------------------------------------------
2022-05-30 20:42:57,205 P

/opt/anaconda3/lib/python3.9/site-packages/flair/trainers/trainer.py:64: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2022-05-30 20:42:58,953 epoch 1 - iter 3/33 - loss 1.01682098 - samples/sec: 55.20 - lr: 0.100000
2022-05-30 20:43:01,894 epoch 1 - iter 6/33 - loss 1.04294419 - samples/sec: 32.66 - lr: 0.100000
2022-05-30 20:43:04,458 epoch 1 - iter 9/33 - loss 1.07340072 - samples/sec: 37.46 - lr: 0.100000
2022-05-30 20:43:07,035 epoch 1 - iter 12/33 - loss 1.04021959 - samples/sec: 37.25 - lr: 0.100000
2022-05-30 20:43:09,553 epoch 1 - iter 15/33 - loss 0.99827320 - samples/sec: 38.15 - lr: 0.100000
2022-05-30 20:43:10,845 epoch 1 - iter 18/33 - loss 1.02313148 - samples/sec: 74.31 - lr: 0.100000
2022-05-30 20:43:17,153 epoch 1 - iter 21/33 - loss 1.03123524 - samples/sec: 15.22 - lr: 0.100000
2022-05-30 20:43:24,792 epoch 1 - iter 24/33 - loss 1.03370120 - samples/sec: 12.57 - lr: 0.100000
2022-05-30 20:43:29,303 epoch 1 - iter 27/33 - loss 1.04273997 - samples/sec: 21.28 - lr: 0.100000
2022-05-30 20:43:37,094 epoch 1 - iter 30/33 - loss 1.04857766 - samples/sec: 12.32 - lr: 0.100000
2022-05-30 20

100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  4.92it/s]

2022-05-30 20:43:40,371 Evaluating as a multi-label problem: False


...

2022-05-30 20:51:20,513 Evaluating as a multi-label problem: False
2022-05-30 20:51:20,534 DEV : loss 0.5127765536308289 - f1-score (micro avg)  0.8324
2022-05-30 20:51:20,540 BAD EPOCHS (no improvement): 0


2022-05-30 20:51:20,542 saving best model
2022-05-30 20:51:21,221 ----------------------------------------------------------------------------------------------------
2022-05-30 20:51:21,222 loading file resources/taggers/example-upos/best-model.pt
2022-05-30 20:51:21,409 SequenceTagger predicts: Dictionary with 44 tags: <unk>, NNP, NN, IN, DT, NNS, CC, ,, JJ, CD, :, ., VBN, VB, TO, VBZ, RB, MD, VBG, VBD, VBP, PRP$, POS, WDT, LS, PRP, NNPS, '', JJR, WRB, JJS, $, ``, EX, -NONE-, RP, RBR, PDT, WP$, RBS, WP, -X-, <START>, <STOP>


100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.02it/s]

2022-05-30 20:51:24,750 Evaluating as a multi-label problem: False
2022-05-30 20:51:24,817 0.8197	0.8197	0.8197	0.8197
2022-05-30 20:51:24,818 
Results:
- F-score (micro) 0.8197
- F-score (macro) 0.4914
- Accuracy 0.8197

By class:
              precision    recall  f1-score   support

          NN     0.6585    0.7569    0.7043      1806
         NNP     0.7638    0.6201    0.6845      1961
          IN     0.8894    0.9592    0.9230      1593
          DT     0.9586    0.9885    0.9733      1570
          CC     0.9750    0.9866    0.9808       673
          JJ     0.6923    0.6934    0.6929       636
           ,     0.9902    0.9728    0.9815       626
          CD     0.8524    0.8569    0.8547       573
         NNS     0.6805    0.6912    0.6858       570
           :     0.7549    0.8770    0.8114       488
           .     0.9786    0.9952    0.9868       413
          VB     0.8144    0.8977    0.8540       391
          TO     0.9784    1.0000    0.9891       363
         VB

{'test_score': 0.8197463768115942,
 'dev_score_history': [0.7715721865400726,
  0.777436470259704,
  0.7782742250767942,
  0.7933538117844178,
  0.8006143535325329,
  0.8159731918458532,
  0.8198827143256073,
  0.8232337335939681,
  0.8151354370287629,
  0.8324490365819603],
 'train_loss_history': [1.0325295742378497,
  0.9888096223147728,
  0.9631876550413973,
  0.9417988501992212,
  0.92350258242295,
  0.8883340689667553,
  0.8744126978643426,
  0.8571559276015643,
  0.841720006612873,
  0.8246402215061988],
 'dev_loss_history': [0.7023018598556519,
  0.6789711117744446,
  0.6370547413825989,
  0.6128749251365662,
  0.606508731842041,
  0.5761227607727051,
  0.5578662753105164,
  0.5433512330055237,
  0.5508217215538025,
  0.5127765536308289]}

In [20]:
# load the model you trained
model = SequenceTagger.load('resources/taggers/example-upos/final-model.pt')

2022-05-30 21:13:52,066 loading file resources/taggers/example-upos/final-model.pt
2022-05-30 21:13:52,263 SequenceTagger predicts: Dictionary with 44 tags: <unk>, NNP, NN, IN, DT, NNS, CC, ,, JJ, CD, :, ., VBN, VB, TO, VBZ, RB, MD, VBG, VBD, VBP, PRP$, POS, WDT, LS, PRP, NNPS, '', JJR, WRB, JJS, $, ``, EX, -NONE-, RP, RBR, PDT, WP$, RBS, WP, -X-, <START>, <STOP>


In [33]:
from utils import *
from textblob import TextBlob
import nltk
from flair.data import Sentence
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/etiennebruno/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
df = df_from_filings()
data = df.text[0]

for sentence in TextBlob(df.text[0]).sentences[20:]: 
    s = Sentence(sentence)
    model.predict(s)
    
    print(s.tag, s.score)
    break

NNS 0.42820659279823303


In [40]:
TextBlob(df.text[0]).sentiment

Sentiment(polarity=0.06307071851174932, subjectivity=0.3252791047736546)

In [43]:
', '.join(df.cik[:100])

'0001553197, 0001553197, 0001553197, 0001553197, 0001553197, 0001553197, 0001553197, 0001553197, 0001553197, 0001553197'